# Main Header and Summary

## Setup and initializations

In [1]:
intention = '''Draft or create a class system to represent the pular entries. 
Ideally this will contain a way to nest entry objects under a root
'''
#%pip install docx
#%pip install python-docx #this mutates docx? 
#%pip install pydantic
#%pip install mypy
# %pip install numpy
from typing import Optional, Dict, List, Any, Union, Tuple
from pydantic import BaseModel, ValidationError, validator, root_validator, Field, constr
import json
from datetime import datetime
from itertools import compress, tee, chain
import re
from collections import Counter
import string
import numpy as np
from verbalexpressions import VerEx
import pickle

import docx
from docx import Document
from pydantic_docx import Docx_Paragraph_and_Runs #type:ignore



In [2]:
import logging
now = datetime.now()
current_time = now.strftime("%Y-%m-%d_-_%H-%M-%S")
logger_filename = f"logs_and_outputs/{current_time}docxFileParse.log"

handler = logging.FileHandler(logger_filename, 'w', 'utf-8') 
handler.setFormatter(logging.Formatter('%(asctime)s %(message)s'))

# logging.setLogRecordFactory(factory)
logging.basicConfig(handlers=[handler], level=logging.DEBUG)
logger = logging.getLogger()

## Docx Pydantic Classes

In [ ]:
##THIS IS NOW A .PY SCRIPT##

## Reading DOCX File and Applying Pydantic Class Parsing

In [10]:
# # %%capture cap --no-stderr

# #Run docx module to parse the docx file
docx_filename = "Fula_Dictionary-repaired.docx"
# docx_filename = "pasted_docx page 1.docx"
document = Document(docx_filename)



char_counts = Counter()

docx_object_list = []
parsed_object_list = []
failed_paras_ind = []
handled_errors = []

for i, para in enumerate(document.paragraphs):
   docx_object_list.append((i,para))
   try:
      entryObj = Docx_Paragraph_and_Runs(**{'paragraph': para, 'paragraph_enumeration': i})
      char_counts.update(entryObj.interogate__para_text())
      parsed_object_list.append((i,entryObj))
   except ValidationError as e:
      suppress = {
            # 'type': ['value_error.any_str.min_length' #ignore zero length run_text, per run validator
            #          ],
            'msg': ['suppressed Validation Error'] #ignore suppressed errors earlier/lower in the stack      
      }
      for err in e.errors():
         if err['msg'] in suppress['msg']:
            handled_errors.append((i,para))
            pass
   except BaseException as e:
      print(e)
      raise
      failed_paras_ind.append((i,para))
      
print('total paras: ',len(docx_object_list))
print('parsed paras: ',len(parsed_object_list))
print('handled errors: ',len(handled_errors))
print('failed paras: ',len(failed_paras_ind))

assert len(docx_object_list) == len(parsed_object_list) + len(handled_errors) + len(failed_paras_ind)


total paras:  32507
parsed paras:  32040
handled errors:  467
failed paras:  0


In [11]:
# with open(output_name, 'w', encoding="utf-8") as f:
#     f.write(cap.stdout)

## Iterating Over Class Objects to extract Dictionary Specific features

### Misc Prep / notes / logger "experiment" input request

In [12]:
# #get current datetime
now = datetime.now()
current_time = now.strftime("%Y-%m-%d_-_%H-%M-%S")

# #create file to save prints (use with jupyter magic enabled at the top of this cell: %%capture cap --no-stderr)
output_name = f"logs_and_outputs/{current_time}_objList_processing_Output.txt"
experiment = input("Enter emperiment description:")
print(f"Experiment time: {current_time}\nExperiment note: {experiment}\n\n")

# logger_filename = f"logs_and_outputs/{current_time}objList_processing.log"

# # Creating an object
# logger = logging.getLogger()

# # Setting the threshold of logger to DEBUG, etc
# logger.setLevel(logging.INFO)

# #add encoding
# handler = logging.FileHandler(logger_filename, 'w', 'utf-8') 
# handler.setFormatter(logging.Formatter('%(asctime)s %(message)s'))
# logger.addHandler(handler) 

Experiment time: 2022-07-29_-_08-15-57
Experiment note: trying this again with more pieces disabled




In [13]:
# # # 2022-07-22 07:20:18,751 paragraph#2731 with text ""   -IR-"" had leading whitespace removed
# # # parsed_object_list = parsed_object_list[2731:3000]
# # # def any(self, value):
# # #    return self.add("([%s])" % value)
# # # def add(self, value):
# # #    if isinstance(value, list):
# # #       self.s.extend(value)
# # #    else:
# # #       self.s.append(value)
# # #    return self
# # low_alph_chars = ''.join([x.lower() for x in char_counts.keys() if x.upper() != x.lower()]) #only uppercase alphabetical chars
# # up_alph_chars = ''.join([x.upper() for x in char_counts.keys() if x.upper() != x.lower()]) #only uppercase alphabetical chars
# # root_note_chars = '-+()? ' #characters that encode the author's notes
# # sub_root_beginnings = '-+('
# # permissive_root_contents = ''.join(list(chain(up_alph_chars,root_note_chars,string.digits)))
# # pattern = '^['+re.escape(sub_root_beginnings)+'][^'+low_alph_chars+']+'
# # print(pattern)
# # print(re.search(pattern = pattern, string = '-KLHFGSLK JEs'))
# # pattern = '^['+re.escape(sub_root_beginnings)+']['+re.escape(permissive_root_contents)+']+'
# # print(pattern)
# # print(re.search(pattern = pattern, string = 'KLHFGSLK JEs'))
# noun_patterns = [r'n\.',r"(?<=\()[^\)]+",r"\/"]
# nounPatternRegex = re.compile('|'.join([p for p in noun_patterns]))
# print(nounPatternRegex)
# s = 'n. dfhjkgqh (wsh) askldjhf / asdfhjkkh'
# print(re.findall(nounPatternRegex,s))
# print(nounPatternRegex.findall(s))

# # if /, the next word is a plural. The plural also then usually has a (el) class after it. commas are multiple version
# # so each lemma noun may have a (class). The lemma may have one or multiple plurals, which will have a modified (class)
# # so have a nouns list? nouns may then be 
# # [('n.',{bool}), #only needed if this gets added to all POS? Or maybe theres information in its use?
# # ([[{word}],{Optional[class]}]),  #singular
#    #([[{word}],{Optional[class]}])]   #plural

# # so nounStruct[0][1] is the flag for noun?
#    # one POS table, and noun will get added there. But the presence of a noun indicates to look here
# # classes may be found [sClass[1] for sClass in nounStruct[1]] will give all classes of singular, [sClass[1] for sClass in nounStruct[1]]

### Iteration and Extraction - Dictionary Specific Logic

In [1]:
%%capture cap 
#--no-stderr

now = datetime.now()
current_time = now.strftime("%Y-%m-%d_-_%H-%M-%S")
print(f"Experiment time: {current_time}\nExperiment note: {experiment}\n\n")
para_text_lookup = {}
root_ind_list = []
subroot_ind_list = []
lemma_ind_list = []
some_error_ind_list = []
reject_ind_list = []
root_and_lemma_one_line = []
root_lookup = {}
lemma_lookup = {}
pos_lookup = {}
pos_ind_list = []

up_alph_chars = [x.upper() for x in char_counts.keys() if x.upper() != x.lower()] #only uppercase alphabetical chars

for i, entryObj in parsed_object_list:

   try:
      para_text_lookup[i] = entryObj.para_text
      successful_cleaner_output:bool = entryObj.cleaner() #by default cleaner removes leading whitespace and merges adjacent runs with identical format features
      # print('sucessful cleaner')
      if not successful_cleaner_output:
         reject_ind_list.append(i)
         print(f'para# {i} IS ONLY whitespace. Need to drop it. #TODO')
   except:
      some_error_ind_list.append(f"cleaner error on p: {i}. Text is {entryObj.interogate__para_text()}")
      print('error on cleaner')
      raise
   try:
      root_note_chars = '-+()? ' #characters that encode the author's notes
      sub_root_beginnings = '-+('
      permissive_root_contents = ''.join(list(chain(up_alph_chars,root_note_chars,string.digits)))

      featureConfig = {
      'root': {'docxFeature': 'run_font_size_pt',
               'strSummary':'fontSize_12.0', 
               'value':12.0,
               # 'text_regex_at_feature': root_expression.compile()
               },
      'subroot': {'docxFeature': 'run_font_size_pt',
               'strSummary':'fontSize_12.0', 
               'value':12.0,
               # 'text_regex_at_feature': subroot_expression.compile()
               },
      'lemma': {'docxFeature': 'run_bold',
               'strSummary':'fontBold', 
               'value':True},
      'lemmaPOS': {'docxFeature': 'run_italic',
               'strSummary':'fontItalic', 
               'value':True},
      }
      is_subroot = False
      # return True, (value_mask, run_text), (regex_mask, regex_matches)
      is_root, (root_mask,run_text), _ = entryObj.single_run_feature_identify(featureConfig['root'])
      if is_root:
         rtext = ''.join(chain(compress(run_text,root_mask))).strip()
         root_lookup[i] = (rtext,root_mask)
         #routine to distinguish between main root and subroot
         for j, r in enumerate(compress(run_text,root_mask)): 
            if j==0:
               # low_alph_chars = ''.join([x.lower() for x in char_counts.keys() if x.upper() != x.lower()]) #only uppercase alphabetical chars
               up_alph_chars = ''.join([x.upper() for x in char_counts.keys() if x.upper() != x.lower()]) #only uppercase alphabetical chars
               root_note_chars = '-+()? ' #characters that encode the author's notes
               sub_root_beginnings = '-+('
               permissive_root_contents = ''.join(list(chain(up_alph_chars,root_note_chars,string.digits)))
               pattern = '^['+re.escape(sub_root_beginnings)+']['+re.escape(permissive_root_contents)+']+'
               m = re.search(pattern = pattern, string = r)
               if m is not None:
                  is_subroot = True
         
         if is_subroot:
            print('\n\nsubroot at para number: ',i)
            paraText = entryObj.interogate__para_text()
            print('\t',paraText)
            subroot_ind_list.append(i)
         else:
            print('\n\nroot at para number: ',i)
            paraText = entryObj.interogate__para_text()
            print('\t',paraText)
            root_ind_list.append(i)
      # return True, (value_mask, run_text), (regex_mask, regex_matches)
      is_lemma, (lemma_mask, run_text), _ = entryObj.single_run_feature_identify(featureConfig['lemma'])
      if is_lemma:
         # print(lemma_mask,run_text)
         ltext = ''.join(chain(compress(run_text,lemma_mask))).strip()
         # print(ltext)
         lemma_lookup[i] = (ltext, lemma_mask)
         # paraText = entryObj.interogate__para_text()
         # print('\t\tp#',i,'\t\t',paraText)
         lemma_ind_list.append(i)
      # is_pos, (pos_mask, run_text), _ = entryObj.single_run_feature_identify(featureConfig['lemma'])
      # if is_pos:
      #    postext = ''.join(chain(compress(run_text,pos_mask))).strip()
      #    # noun_patterns = [r'n\.',r"(?<=\()[^\)]+",r"\/"]
      #    # nounPatternRegex = re.compile('|'.join([p for p in noun_patterns]))
      #    pos_lookup[i] = (postext,pos_mask)
      #    pos_ind_list.append(i)
      #    if is_pos and not is_lemma:
      #       entryObj.paragraph_logger(level = 10,msg = f'Unexpected structure:: para#{i} has a POS: {postext} but this para does NOT have a lemma. last lemma at para#{lemma_ind_list[-1]} was "{lemma_lookup[lemma_ind_list[-1]]}"',print_bool = True)
      # if is_pos and (is_root or is_subroot):
      if is_lemma and is_root:
         print(f'this para# {i} has BOTH lemma AND root')
         root_and_lemma_one_line.append(i)

   except BaseException as e:
      
      some_error_ind_list.append(i)
      raise e

print('total paras: ',len(docx_object_list))
print('parsed paras: ',len(parsed_object_list))
print('handled errors: ',len(handled_errors))
print('failed paras: ',len(failed_paras_ind))

assert len(docx_object_list) == len(parsed_object_list) + len(handled_errors) + len(failed_paras_ind)

print('roots: ',len(root_ind_list))
print('subroots: ',len(subroot_ind_list))
print('lemmas: ',len(lemma_ind_list))
print('root_and_lemma_one_line: ',len(root_and_lemma_one_line))
print('additional cleaner rejects: ',len(reject_ind_list))
print('additional error rejects: ',len(some_error_ind_list))

print('num entities: ',len(root_ind_list) + len(lemma_ind_list) + len(subroot_ind_list))
num_good_paras_of_other_content= len(root_ind_list) + len(lemma_ind_list) - len(root_and_lemma_one_line) + len(subroot_ind_list)\
                                    + len(reject_ind_list) + len(some_error_ind_list)
print('num_good_paras_of_other_content: ',num_good_paras_of_other_content)
# # Test messages
logger.debug("logger debug test")
logger.info("Just an information")
# logger.warning("Its a Warning")
# logger.error("Did you try to divide by zero")
# logger.critical("Internet is down")

SyntaxError: invalid syntax (1010491370.py, line 91)

In [15]:
with open(output_name, 'w', encoding="utf-8") as f:
    f.write(cap.stdout)


# Open a file and use dump()
with open('parsed_objectClass_outcomes_dict.pkl', 'wb') as file:
    # A new file will be created
    outcomes_dict = {}
    outcomes_dict['parsed_object_list'] = parsed_object_list
    outcomes_dict['para_text_lookup'] = para_text_lookup
    outcomes_dict['root_ind_list'] = root_ind_list
    outcomes_dict['subroot_ind_list'] = subroot_ind_list
    outcomes_dict['lemma_ind_list'] = lemma_ind_list
    outcomes_dict['root_and_lemma_one_line'] = root_and_lemma_one_line
    outcomes_dict['root_lookup'] = root_lookup
    outcomes_dict['lemma_lookup'] = lemma_lookup
    # outcomes_dict['pos_lookup'] = pos_lookup
    outcomes_dict['char_counts'] = char_counts
    pickle.dump(outcomes_dict, file)